In [1]:
#%pip install cufflinks
#%pip install configparser
#%pip install pyecharts
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dataDealer import getData 
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output

df = getData()

"""
ydf = df.sort_values(["Year"]).reset_index(drop=True)
    
x = ydf["Year"]
y = ydf["Weight"]
"""

columns = {"MaxPlayTime" : 'MaxPlayTime', 
           "MinPlayTime" : 'MinPlayTime',
           "MaxPlayers" : 'MaxPlayers',
           "MinPlayers" : 'MinPlayers',
           "BestPlayers" : 'BestPlayers',
           "MinAge" : 'MinAge',
           "Weight" : 'Weight',
           "Year":'Year'
          }



@interact
def show_articles_more_than(column= columns, x=(0,500)): 
    print(f"符合條件的桌遊有 {len(df.loc[df[column] > x])} 款")
    return df.loc[df[column] > x]


[('address', 'localhost'), ('port', '3306'), ('account', 'root'), ('password', '00000000'), ('db', 'mydb'), ('table', 'boardgame')]
MySQL


interactive(children=(Dropdown(description='column', options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTime': 'Mi…

In [7]:


def update_annot(ind):
    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    tempList = ind["ind"]
    #text = "{}".format(" ".join(tempList))
    text = ydf['Name'][int(tempList)]
    
    annot.set_text(text)
    #annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
    #annot.get_bbox_patch().set_alpha(0.4)


def hover(event):
    
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()

@interact(col1 = columns, col2 = columns)
def setData(col1= 'Year', col2= 'Weight'):
    global ydf
    ydf = df.sort_values([col1]).reset_index(drop=True)
    
    x = ydf[col1]
    y = ydf[col2]

    global ax
    fig,ax = plt.subplots()
    global sc
    sc = plt.scatter(x, y)
    global annot
    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                  bbox=dict(boxstyle="round", fc="w"),
#                    arrowprops=dict(arrowstyle="->")
                   )
    annot.set_visible(False)
    
    fig.canvas.mpl_connect("motion_notify_event", hover)

    plt.show()


#clear_output(wait=True)

interactive(children=(Dropdown(description='col1', index=7, options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTim…

In [6]:
from pyecharts import options as opts
from pyecharts.charts import Pie

@interact()
def rosePie(col1 = columns):
    data = df[col1].value_counts()
    v = data.index
    c = (
        Pie()
        .add(
            "123",
            [list(z) for z in zip(v, list(data))],
            radius=["50%", "80%"],
            center=["50%", "50%"],
            rosetype="area",
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="玫瑰圖展示"))

    )
    return c.render_notebook()
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))

interactive(children=(Dropdown(description='col1', options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTime': 'MinP…

interactive(children=(Dropdown(description='col1', options={'MaxPlayTime': 'MaxPlayTime', 'MinPlayTime': 'MinP…

In [4]:
import pyecharts.options as opts
from pyecharts.charts import Bar3D

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.apache.org/examples/editor.html?c=bar3d-punch-card&gl=1
目前无法实现的功能:
1、光照和阴影暂时无法设置
"""

hours = [
    "12a",
    "1a",
    "2a",
    "3a",
    "4a",
    "5a",
    "6a",
    "7a",
    "8a",
    "9a",
    "10a",
    "11a",
    "12p",
    "1p",
    "2p",
    "3p",
    "4p",
    "5p",
    "6p",
    "7p",
    "8p",
    "9p",
    "10p",
    "11p",
]
days = ["Saturday", "Friday", "Thursday", "Wednesday", "Tuesday", "Monday", "Sunday"]

data = [
    [0, 0, 5],
    [0, 1, 1],
    [0, 2, 0],
    [0, 3, 0],
    [0, 4, 0],
    [0, 5, 0],
    [0, 6, 0],
    [0, 7, 0],
    [0, 8, 0],
    [0, 9, 0],
    [0, 10, 0],
    [0, 11, 2],
    [0, 12, 4],
    [0, 13, 1],
    [0, 14, 1],
    [0, 15, 3],
    [0, 16, 4],
    [0, 17, 6],
    [0, 18, 4],
    [0, 19, 4],
    [0, 20, 3],
    [0, 21, 3],
    [0, 22, 2],
    [0, 23, 5],
    [1, 0, 7],
    [1, 1, 0],
    [1, 2, 0],
    [1, 3, 0],
    [1, 4, 0],
    [1, 5, 0],
    [1, 6, 0],
    [1, 7, 0],
    [1, 8, 0],
    [1, 9, 0],
    [1, 10, 5],
    [1, 11, 2],
    [1, 12, 2],
    [1, 13, 6],
    [1, 14, 9],
    [1, 15, 11],
    [1, 16, 6],
    [1, 17, 7],
    [1, 18, 8],
    [1, 19, 12],
    [1, 20, 5],
    [1, 21, 5],
    [1, 22, 7],
    [1, 23, 2],
    [2, 0, 1],
    [2, 1, 1],
    [2, 2, 0],
    [2, 3, 0],
    [2, 4, 0],
    [2, 5, 0],
    [2, 6, 0],
    [2, 7, 0],
    [2, 8, 0],
    [2, 9, 0],
    [2, 10, 3],
    [2, 11, 2],
    [2, 12, 1],
    [2, 13, 9],
    [2, 14, 8],
    [2, 15, 10],
    [2, 16, 6],
    [2, 17, 5],
    [2, 18, 5],
    [2, 19, 5],
    [2, 20, 7],
    [2, 21, 4],
    [2, 22, 2],
    [2, 23, 4],
    [3, 0, 7],
    [3, 1, 3],
    [3, 2, 0],
    [3, 3, 0],
    [3, 4, 0],
    [3, 5, 0],
    [3, 6, 0],
    [3, 7, 0],
    [3, 8, 1],
    [3, 9, 0],
    [3, 10, 5],
    [3, 11, 4],
    [3, 12, 7],
    [3, 13, 14],
    [3, 14, 13],
    [3, 15, 12],
    [3, 16, 9],
    [3, 17, 5],
    [3, 18, 5],
    [3, 19, 10],
    [3, 20, 6],
    [3, 21, 4],
    [3, 22, 4],
    [3, 23, 1],
    [4, 0, 1],
    [4, 1, 3],
    [4, 2, 0],
    [4, 3, 0],
    [4, 4, 0],
    [4, 5, 1],
    [4, 6, 0],
    [4, 7, 0],
    [4, 8, 0],
    [4, 9, 2],
    [4, 10, 4],
    [4, 11, 4],
    [4, 12, 2],
    [4, 13, 4],
    [4, 14, 4],
    [4, 15, 14],
    [4, 16, 12],
    [4, 17, 1],
    [4, 18, 8],
    [4, 19, 5],
    [4, 20, 3],
    [4, 21, 7],
    [4, 22, 3],
    [4, 23, 0],
    [5, 0, 2],
    [5, 1, 1],
    [5, 2, 0],
    [5, 3, 3],
    [5, 4, 0],
    [5, 5, 0],
    [5, 6, 0],
    [5, 7, 0],
    [5, 8, 2],
    [5, 9, 0],
    [5, 10, 4],
    [5, 11, 1],
    [5, 12, 5],
    [5, 13, 10],
    [5, 14, 5],
    [5, 15, 7],
    [5, 16, 11],
    [5, 17, 6],
    [5, 18, 0],
    [5, 19, 5],
    [5, 20, 3],
    [5, 21, 4],
    [5, 22, 2],
    [5, 23, 0],
    [6, 0, 1],
    [6, 1, 0],
    [6, 2, 0],
    [6, 3, 0],
    [6, 4, 0],
    [6, 5, 0],
    [6, 6, 0],
    [6, 7, 0],
    [6, 8, 0],
    [6, 9, 0],
    [6, 10, 1],
    [6, 11, 0],
    [6, 12, 2],
    [6, 13, 1],
    [6, 14, 3],
    [6, 15, 4],
    [6, 16, 0],
    [6, 17, 0],
    [6, 18, 0],
    [6, 19, 0],
    [6, 20, 1],
    [6, 21, 2],
    [6, 22, 2],
    [6, 23, 6],
]
data = [[d[1], d[0], d[2]] for d in data]


(
    Bar3D()
    .add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(type_="category", data=hours),
        yaxis3d_opts=opts.Axis3DOpts(type_="category", data=days),
        zaxis3d_opts=opts.Axis3DOpts(type_="value"),
    )
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(
            max_=20,
            range_color=[
                "#313695",
                "#4575b4",
                "#74add1",
                "#abd9e9",
                "#e0f3f8",
                "#ffffbf",
                "#fee090",
                "#fdae61",
                "#f46d43",
                "#d73027",
                "#a50026",
            ],
        )
    )
    .render_notebook()
)